In [1]:
import sqlite3
import pandas as pd
import datetime
import seaborn as sns
from collections import Counter
from tqdm import tqdm

sns.set_style('white')
sns.set_context('notebook')

# 1. Read in SQL Databases

## Autistic Cohort

In [2]:
conn1 = sqlite3.connect('./Data/knwlg_blf.db')
df1 = pd.read_sql("SELECT * "
                 "FROM subjects s "
                 "JOIN trials t "
                 "ON s.prolific_id=t.prolific_id "
                 "JOIN demographics d "
                 "ON s.prolific_id=d.prolific_id "
                 "JOIN autism_scores a "
                 "ON s.prolific_id=a.prolific_id "
                 "WHERE s.completion_code LIKE '%XXX%' "
                 "AND s.recaptcha_complete=1 "
                 "AND t.trial_type='test'", con=conn1)

#df1 = df1.loc[~df1.target_onset.isnull()]
#df1 = df1.loc[~df1.AQ_rating_1.isnull()] # this is because 5 ids are in DB multiple times somehow
#df1 = df1.loc[~df1.age.isnull()] # this is because 5 ids are in DB multiple times somehow

df1 = df1.loc[:,~df1.columns.duplicated()].copy() # prolific id from both tables will be in df, remove duplicate

weird = {}
for subj in df1.prolific_id.unique():
    weird[subj] = len(df1.loc[df1.prolific_id == subj])
strange = [key  for (key, value) in weird.items() if value != 12] # these subjects have duplicate rows with no data
df1 = df1.loc[~df1.prolific_id.isin(strange)]

fel_a = pd.read_sql("SELECT * "
                    "FROM felicities f "
                    "JOIN subjects s "
                    "ON s.prolific_id=f.prolific_id "
                    "WHERE s.completion_code LIKE '%XXX%' "
                    "AND s.recaptcha_complete=1 ", con=conn1)
fel_a = fel_a.loc[:,~fel_a.columns.duplicated()].copy() # prolific id from both tables will be in df, remove duplicate
fel_a = fel_a.reset_index()

# Add Felicity rating to appropriate row
df1['felicity_rating'] = 999
for pp in tqdm(df1.prolific_id.unique()):
    for i, trl in df1.loc[df1.prolific_id == pp].iterrows():
        df1.loc[(df1.prolific_id == pp) & (df1.trial_num == trl.trial_num), 'felicity_rating'] = fel_a.loc[(fel_a.prolific_id == pp) & (fel_a.block1_trial_num == trl.trial_num)].felicity_rating.values[0]

100%|██████████| 617/617 [00:18<00:00, 32.83it/s]


## Correcting Responses and RT in autism group

In [3]:
def convert_time(x):
    if type(x) == str:
        return datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f')

df1[['response_onset','target_onset','participation_date','completion_time']] = df1[['response_onset','target_onset','participation_date','completion_time']].applymap(convert_time)

df1['rt'] = df1.apply(lambda row: row['response_onset'] - row['target_onset'], axis=1)
df1['rt_ms'] = df1.apply(lambda row: int(round(row['rt'].total_seconds() * 1e3)), axis=1)

In [4]:
df1['response_key'] = df1.response_key.apply(eval)
df1['keypress_time'] = [eval(x) for x in df1.keypress_time]
df1 = df1[[xx == list for xx in [type(x) for x in df1.response_key]]].reset_index(drop=True) # make sure responses are saved as a list.
# If subjects hit more than one key, use only the last one
df1['full_response'] = df1['response_key']
df1['response_key'] = [resp[-1].lower() for resp in df1.response_key]
df1['corrected_resp'] = df1.response_key
df1['corrected_rt'] = df1.rt_ms


In [5]:
for i, trl in df1[df1.apply(lambda row: len(row['full_response']) > 1, axis=1)].iterrows(): # iterate through rows where >1  response was made
    keys_pressed = list(Counter(trl.full_response).keys()) # list the unique keys, in the temporal order they were pressed. fjf -> fj
    if '999' in keys_pressed: # if the timer hit 5 seconds
        if keys_pressed.index('999') == 0: # and there were no other keys pressed before then
            intended_answer = '999' # their answer is recorded as 999
        else : # if there were other keys pressed before the 5-second timer (but the 5 seconds  was still reached because of internet transfer)
            intended_answer = keys_pressed[:keys_pressed.index('999')][-1] # the last key pressed before the 999 was the intended response
    else:     # If there is no timeout, take the first instance of the last key pressed.
        kp = [x for x in keys_pressed if x =='f'or x== 'j']
        if len(kp) > 0:
            intended_answer = kp[-1]
        else:
            intended_answer = keys_pressed[-1]

    onset = trl.keypress_time[trl.full_response.index(intended_answer)] #onset of the first instance of the last key pressed.
    rt = int((onset - trl.target_onset).total_seconds() * 1e3)  # compute the RT for that key press
    df1.loc[df1.id == trl.id, ['response_key', 'rt_ms', 'response_onset']] = intended_answer, rt, onset

# SUSPICIOUS EMAILS
df1 = df1[["mailfence" not in x for x in df1.email]].reset_index(drop=True)


## Neurotypical Cohort

In [6]:
conn2 = sqlite3.connect('./Data/knwlg_blf_controls.db')
df2 = pd.read_sql("SELECT * "
                 "FROM subjects s "
                 "JOIN trials t "
                 "ON s.prolific_id=t.prolific_id "
                 "JOIN demographics d "
                 "ON s.prolific_id=d.prolific_id "
                 "JOIN autism_scores a "
                 "ON s.prolific_id=a.prolific_id "
                 "WHERE s.completion_code LIKE '%548DA3BD%' "
                 "AND t.trial_type='test'", con=conn2)

df2 = df2.loc[~df2.target_onset.isnull()]
df2 = df2.loc[~df2.AQ_rating_1.isnull()] # this is because 5 ids are in DB multiple times somehow

df2 = df2.loc[:,~df2.columns.duplicated()].copy() # prolific id from both tables will be in df, remove duplicate

fel_b = pd.read_sql("SELECT * "
                    "FROM felicities f "
                    "JOIN subjects s "
                    "ON s.prolific_id=f.prolific_id "
                    "WHERE s.completion_code LIKE '%548DA3BD%' ", con=conn2)
fel_b = fel_b.loc[:,~fel_b.columns.duplicated()].copy() # prolific id from both tables will be in df, remove duplicate
fel_b = fel_b.reset_index()

# Add Felicity rating to appropriate row
df2['felicity_rating'] = 999
for pp in tqdm(df2.prolific_id.unique()):
    for i, trl in df2.loc[df2.prolific_id == pp].iterrows():
        df2.loc[(df2.prolific_id == pp) & (df2.trial_num == trl.trial_num), 'felicity_rating'] = fel_b.loc[(fel_b.prolific_id == pp) & (fel_b.block1_trial_num == trl.trial_num)].felicity_rating.values[0]

100%|██████████| 501/501 [00:13<00:00, 37.39it/s]


In [7]:
df2['response_key'] = df2.response_key.apply(eval)
df2['response_key'] = df2.apply(lambda row: row['response_key'][-1].lower(), axis=1)
df2[['response_onset','target_onset','participation_date']] = df2[['response_onset','target_onset','participation_date']].applymap(convert_time)

df2['rt'] = df2.apply(lambda row: row['response_onset'] - row['target_onset'], axis=1)
df2['rt_ms'] = df2.apply(lambda row: int(round(row['rt'].total_seconds() * 1e3)), axis=1)

## Getting Columns to match

In [10]:
df1['group'] = 'autism'
df2['group'] = 'neurotypical'
df2['diag'] = 'n/a'
COI = ['prolific_id', 'participation_date', 'GMT_timestamp', 'trial_num', 'correct', 'scenario', 'belief_type', 'ascription_type','correct_answer', 'rt_ms','response_key', 'age', 'gender', 'ethnicity', 'education', 'AQ_rating_1', 'AQ_rating_2', 'AQ_rating_3', 'AQ_rating_4', 'AQ_rating_5', 'AQ_rating_6', 'AQ_rating_7','AQ_rating_8', 'AQ_rating_9', 'AQ_rating_10', 'felicity_rating', 'group', 'diag']
df1 = df1[sorted(COI)]

df2 = df2[sorted(COI)]

## Combining Cohorts

In [11]:
# columns in autism group that are not in controls
if len(df1.columns[[x not in df2.columns for x in df1.columns]].tolist())== 0:
    dat = pd.concat([df1, df2], axis=0)
if (len(df1.prolific_id.unique()) + len(df2.prolific_id.unique())) == len(dat.prolific_id.unique()):

    dat['Infelicity Rating'] = 8 - dat.felicity_rating
    dat.rename(columns={'rt_ms': 'rt', 'ascription_type': 'ascription', 'belief_type': 'agent_state', 'Infelicity Rating': 'infelicity', 'prolific_id':'subject', 'scenario':'item'}, inplace=True)

    for old, new in {'IG':'Ignorance', 'TB':'True Info', 'FB':'False Info'}.items():
        dat.agent_state.replace(old, new, inplace=True)


# Exclusions

## Timeouts

In [12]:
dat['timeout'] = False
dat.loc[dat.response_key == '999', 'timeout'] = True

## Compute Accuracy

In [13]:
#  correct answers
dat.loc[dat.agent_state == 'True Info', 'correct_answer'] = 'j'
dat.loc[dat.agent_state != 'True Info', 'correct_answer'] = 'f'
# update correct column based on new answer key
dat['correct'] = dat.apply(lambda row: row['response_key'] == row['correct_answer'], axis=1)

dat.loc[dat.timeout == True, 'correct'] = False # timeouts are considered incorrect

threshold = .67
dat[['accurate', 'too_fast', 'too_slow', 'in_sample']] = False

for subj in tqdm(dat.subject.unique()):
    sdat = dat.loc[dat.subject == subj]
    if len(sdat) != 12:
        print('ohshit')
        break
    if sdat.correct.mean() >= threshold:
        dat.loc[dat.subject == subj, 'accurate'] = True
    if sdat.rt.mean() < 1000:
        dat.loc[dat.subject == subj, 'too_fast'] = True
    if sdat.rt.mean() > 4000:
        dat.loc[dat.subject == subj, 'too_slow'] = True


dat.loc[(dat.accurate == True) & (dat.too_fast == False) & (dat.too_slow == False), 'in_sample'] = True

100%|██████████| 1058/1058 [00:02<00:00, 393.28it/s]


## Trial Exclusions

In [14]:
dat.loc[(dat.rt < 1000) | (dat.rt > 4500), 'in_sample'] = False


# Autism Quotient

In [15]:
## Scoring
agree = ['AQ_rating_1', 'AQ_rating_7', 'AQ_rating_8', 'AQ_rating_10'] # score of 3 or 4 get a point
disagree = ['AQ_rating_2','AQ_rating_3','AQ_rating_4','AQ_rating_5','AQ_rating_6','AQ_rating_9', ] # score of 1 or 2 get a point]
a = dat[agree] >= 3
b = dat[disagree] < 3
dat['AQ_score'] = a.sum(axis=1) + b.sum(axis=1)
dat.drop(['AQ_rating_'+str(i) for i in range(1, 11)], inplace=True, axis=1)


# Save the clean file

In [16]:
dat.to_csv('./Data/full_sample_clean.csv', sep=',', header=True)